In [127]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import time 

import torch
import mrmr
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [128]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'uci-data/'
MODELS_PATH = DATA_PATH + 'models/raw-models/'
FEATURES = ['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]
SAMPLE_SIZE = 10000

In [129]:
# Load the data
train_set = pd.read_csv(DATA_PATH + "self-calculated/raw-train-data.txt", sep='\s+', header=None)
test_set = pd.read_csv(DATA_PATH + "self-calculated/raw-test-data.txt", sep='\s+', header=None)

train_set.columns=FEATURES
test_set.columns=FEATURES

In [130]:
# Creating model
device = None

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

class RawDataModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(7, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(128, 32),
            nn.SiLU(),
            nn.Linear(32, 8),
            nn.Softmax()  # or nn.Softmax(dim=1) for multi-class classification
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
model = RawDataModel().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [131]:
# Training and testing function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [132]:
# Dataset preparation
class HAPTDataset(Dataset):
    def __init__(self, dataset, features, label):
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)[:SAMPLE_SIZE]
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)[:SAMPLE_SIZE]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
train_dataset = HAPTDataset(train_set, train_set.columns[:-1], 'activity')
test_dataset = HAPTDataset(test_set, test_set.columns[:-1], 'activity')

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [133]:
epochs = 5
perf_timer = time.perf_counter()
perf_acc = ""

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    perf_acc = test(test_dataloader, model, loss_function)
    
print(perf_acc)
perf_timer = time.perf_counter() - perf_timer
print("Done!")

Epoch 1
-------------------------------
loss: 1.822390  [    8/10000]
loss: 2.091010  [  808/10000]
loss: 1.571857  [ 1608/10000]
loss: 2.068745  [ 2408/10000]
loss: 1.994656  [ 3208/10000]
loss: 1.966123  [ 4008/10000]
loss: 1.979111  [ 4808/10000]
loss: 1.399224  [ 5608/10000]
loss: 2.079567  [ 6408/10000]
loss: 1.834152  [ 7208/10000]
loss: 1.849369  [ 8008/10000]
loss: 1.917994  [ 8808/10000]
loss: 1.692412  [ 9608/10000]
Epoch 2
-------------------------------
loss: 1.713806  [    8/10000]
loss: 1.498964  [  808/10000]
loss: 1.529834  [ 1608/10000]
loss: 1.625329  [ 2408/10000]
loss: 1.892533  [ 3208/10000]
loss: 1.459074  [ 4008/10000]
loss: 1.788037  [ 4808/10000]
loss: 1.514153  [ 5608/10000]
loss: 1.990882  [ 6408/10000]
loss: 1.653336  [ 7208/10000]
loss: 1.866607  [ 8008/10000]
loss: 1.492836  [ 8808/10000]
loss: 1.726650  [ 9608/10000]
Epoch 3
-------------------------------
loss: 1.710512  [    8/10000]
loss: 1.787637  [  808/10000]
loss: 1.719837  [ 1608/10000]
loss: 1.91

In [134]:
model_name = "cnn_2"
if True:
    torch.save(model.state_dict(), MODELS_PATH + model_name + ".pth")
    print("Saved PyTorch Model State to model.pth")

    with open(MODELS_PATH + model_name + ".txt", "w") as f:
        f.write("Epochs: {}\n".format(epochs))
        f.write("Feature Selection: {}\n".format("MRMR"))
        f.write("Feature Set: {}\n".format(FEATURES))
        f.write("Model: {}\n".format(str(model)))
        f.write("Loss Function: {}\n".format("Cross Entropy Loss"))
        f.write("Optimizer: {}\n\n\n\n".format(str(optimizer)))
        f.write("Results: {}\n".format(perf_acc))
        f.write("Timer: {}s\n".format(round(perf_timer, 2)))

Saved PyTorch Model State to model.pth
